In [42]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from multiprocessing import  Pool    #split the worklod into multiple CPU cores

In [43]:
def most_frequent(s):
    return s.value_counts().index[0]

def least_frequent(s):
    return s.value_counts().index[-1]

def most_frequent_count(s):
    return s.value_counts().values[0]

def least_frequent_count(s):
    return s.value_counts().values[-1]

def parallelize_dataframe(df, func, n_cores=4):
    df_split = np.array_split(df, n_cores)
    pool = Pool(n_cores)
    df = pd.concat(pool.map(func, df_split))
    pool.close()
    pool.join()
    return df

# Data Cleaning

### Item Information

In [44]:
item_data = pd.read_csv(r'C:\Users\sahil\OneDrive\Pictures\Documents\OneDrive\Desktop\Project\Data\item_data.csv')
item_data.head()

,item_id,brand,brand_type,category
0,1,1,Established,Grocery
1,2,1,Established,Miscellaneous
2,3,56,Local,Bakery
3,4,56,Local,Grocery
4,5,56,Local,Grocery


In [45]:
item_data.isnull().sum()   # no null values in Dataset

item_id       0
brand         0
brand_type    0
category      0
dtype: int64

In [46]:
item_data['brand_type'].value_counts()

brand_type
Established    62842
Local          11224
Name: count, dtype: int64

as we can see brand_type have only two classes let them encode 

In [47]:

item_data["brand_type"]=item_data["brand_type"].map({'Established':1,'Local':0})

In [48]:
item_data

,item_id,brand,brand_type,category
0,1,1,1,Grocery
1,2,1,1,Miscellaneous
2,3,56,0,Bakery
3,4,56,0,Grocery
4,5,56,0,Grocery
...,...,...,...,...
74061,74062,5490,1,Pharmaceutical
74062,74063,5497,1,Pharmaceutical
74063,74064,5497,1,Pharmaceutical
74064,74065,5520,1,Pharmaceutical


In [49]:
item_data.isnull().sum()

item_id       0
brand         0
brand_type    0
category      0
dtype: int64

In [50]:
item_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 74066 entries, 0 to 74065
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   item_id     74066 non-null  int64 
 1   brand       74066 non-null  int64 
 2   brand_type  74066 non-null  int64 
 3   category    74066 non-null  object
dtypes: int64(3), object(1)
memory usage: 2.3+ MB


### Campaign Information

In [51]:
campaign_data = pd.read_csv(r'C:\Users\sahil\OneDrive\Pictures\Documents\OneDrive\Desktop\Project\Data\campaign_data.csv', parse_dates=['start_date', 'end_date'], dayfirst=True)
campaign_data.head()

C:\Users\sahil\AppData\Local\Temp\ipykernel_27384\708254959.py:1: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  campaign_data = pd.read_csv(r'C:\Users\sahil\OneDrive\Pictures\Documents\OneDrive\Desktop\Project\Data\campaign_data.csv', parse_dates=['start_date', 'end_date'], dayfirst=True)
C:\Users\sahil\AppData\Local\Temp\ipykernel_27384\708254959.py:1: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  campaign_data = pd.read_csv(r'C:\Users\sahil\OneDrive\Pictures\Documents\OneDrive\Desktop\Project\Data\campaign_data.csv', parse_dates=['start_date', 'end_date'], dayfirst=True)


,campaign_id,campaign_type,start_date,end_date
0,24,Y,2013-10-21,2013-12-20
1,25,Y,2013-10-21,2013-11-22
2,20,Y,2013-09-07,2013-11-16
3,23,Y,2013-10-08,2013-11-15
4,21,Y,2013-09-16,2013-10-18


In [52]:
campaign_data.isnull().sum()   # no null values in Dataset

campaign_id      0
campaign_type    0
start_date       0
end_date         0
dtype: int64

In [53]:
campaign_data['campaign_type'].value_counts()


campaign_type
Y    22
X     6
Name: count, dtype: int64

In [54]:
campaign_data['campaign_type'] = campaign_data['campaign_type'].map({'X': 0, 'Y': 1})

we have start date and end date of campaign lets add another column of duration of each campaign

In [55]:
campaign_data['duration'] = (campaign_data['end_date'] - campaign_data['start_date']).dt.days
# dt is datetime accessor in pandas through which we can access the date and time attributes of the series.

In [56]:
campaign_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 28 entries, 0 to 27
Data columns (total 5 columns):
 #   Column         Non-Null Count  Dtype         
---  ------         --------------  -----         
 0   campaign_id    28 non-null     int64         
 1   campaign_type  28 non-null     int64         
 2   start_date     28 non-null     datetime64[ns]
 3   end_date       28 non-null     datetime64[ns]
 4   duration       28 non-null     int64         
dtypes: datetime64[ns](2), int64(3)
memory usage: 1.2 KB


In [57]:
campaign_data_index = campaign_data.set_index('campaign_id')
campaign_data_index.info()

<class 'pandas.core.frame.DataFrame'>
Index: 28 entries, 24 to 26
Data columns (total 4 columns):
 #   Column         Non-Null Count  Dtype         
---  ------         --------------  -----         
 0   campaign_type  28 non-null     int64         
 1   start_date     28 non-null     datetime64[ns]
 2   end_date       28 non-null     datetime64[ns]
 3   duration       28 non-null     int64         
dtypes: datetime64[ns](2), int64(2)
memory usage: 1.1 KB


### Customer Information

In [58]:
customer_data = pd.read_csv(r'C:\Users\sahil\OneDrive\Pictures\Documents\OneDrive\Desktop\Project\Data\customer_demographics.csv')
customer_data.head()

,customer_id,age_range,marital_status,rented,family_size,no_of_children,income_bracket
0,1,70+,Married,0,2,NaN,4
1,6,46-55,Married,0,2,NaN,5
2,7,26-35,NaN,0,3,1,3
3,8,26-35,NaN,0,4,2,6
4,10,46-55,Single,0,1,NaN,5


In [59]:
customer_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 760 entries, 0 to 759
Data columns (total 7 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   customer_id     760 non-null    int64 
 1   age_range       760 non-null    object
 2   marital_status  431 non-null    object
 3   rented          760 non-null    int64 
 4   family_size     760 non-null    object
 5   no_of_children  222 non-null    object
 6   income_bracket  760 non-null    int64 
dtypes: int64(3), object(4)
memory usage: 41.7+ KB


In [60]:
customer_data.isnull().sum()

customer_id         0
age_range           0
marital_status    329
rented              0
family_size         0
no_of_children    538
income_bracket      0
dtype: int64

In [61]:
for col in customer_data.columns:
    null_count = customer_data[col].isnull().sum()
    null_percent = (null_count / len(customer_data))*100
    print(f"null_percentage of {col}: {null_percent:.2f}%")


null_percentage of customer_id: 0.00%
null_percentage of age_range: 0.00%
null_percentage of marital_status: 43.29%
null_percentage of rented: 0.00%
null_percentage of family_size: 0.00%
null_percentage of no_of_children: 70.79%
null_percentage of income_bracket: 0.00%


we have almost 43% mull values in marital_status column and 70% null values in family_size but as per our problem statement this are very important attributes.
similarly we can see that 'marital_status', 'family_size', 'no_of_children' this are inter related features lets do something with them

Filling the missing values in customer informations

In [62]:
def get_marital_status(row):
    na_row = row.isna()    #checks for missing values (NaNs) in a row of a DataFrame
    if not na_row['marital_status']:
        return row['marital_status']
    return 'Married' if row['family_size'] - row['no_of_children'] > 1 else 'Single'

customer_data['family_size'] = customer_data['family_size'].str.replace('+','').astype('int')
customer_data['no_of_children'] = customer_data['no_of_children'].fillna('0').str.replace('+','').astype('int')
customer_data['marital_status'] = customer_data.apply(get_marital_status, axis=1)
customer_data['marital_status'] = customer_data['marital_status'].replace({'Single': 0, 'Married': 1})

customer_data.head()

,customer_id,age_range,marital_status,rented,family_size,no_of_children,income_bracket
0,1,70+,1,0,2,0,4
1,6,46-55,1,0,2,0,5
2,7,26-35,1,0,3,1,3
3,8,26-35,1,0,4,2,6
4,10,46-55,0,0,1,0,5


In [63]:
customer_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 760 entries, 0 to 759
Data columns (total 7 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   customer_id     760 non-null    int64 
 1   age_range       760 non-null    object
 2   marital_status  760 non-null    int64 
 3   rented          760 non-null    int64 
 4   family_size     760 non-null    int32 
 5   no_of_children  760 non-null    int32 
 6   income_bracket  760 non-null    int64 
dtypes: int32(2), int64(4), object(1)
memory usage: 35.8+ KB


### Customer Transactions

In [64]:
transaction_data = pd.read_csv(r'C:\Users\sahil\OneDrive\Pictures\Documents\OneDrive\Desktop\Project\Data\customer_transaction_data.csv', parse_dates=['date'])
transaction_data.head()

,date,customer_id,item_id,quantity,selling_price,other_discount,coupon_discount
0,2012-01-02,1501,26830,1,35.26,-10.69,0.0
1,2012-01-02,1501,54253,1,53.43,-13.89,0.0
2,2012-01-02,1501,31962,1,106.50,-14.25,0.0
3,2012-01-02,1501,33647,1,67.32,0.00,0.0
4,2012-01-02,1501,48199,1,71.24,-28.14,0.0


In [65]:
transaction_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1324566 entries, 0 to 1324565
Data columns (total 7 columns):
 #   Column           Non-Null Count    Dtype         
---  ------           --------------    -----         
 0   date             1324566 non-null  datetime64[ns]
 1   customer_id      1324566 non-null  int64         
 2   item_id          1324566 non-null  int64         
 3   quantity         1324566 non-null  int64         
 4   selling_price    1324566 non-null  float64       
 5   other_discount   1324566 non-null  float64       
 6   coupon_discount  1324566 non-null  float64       
dtypes: datetime64[ns](1), float64(3), int64(3)
memory usage: 70.7 MB


### Coupon Item Mapping

In [66]:
coupon_item = pd.read_csv(r'C:\Users\sahil\OneDrive\Pictures\Documents\OneDrive\Desktop\Project\Data\coupon_item_mapping.csv')
coupon_item.head()

,coupon_id,item_id
0,105,37
1,107,75
2,494,76
3,522,77
4,518,77


In [67]:
coupon_item.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 92663 entries, 0 to 92662
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype
---  ------     --------------  -----
 0   coupon_id  92663 non-null  int64
 1   item_id    92663 non-null  int64
dtypes: int64(2)
memory usage: 1.4 MB


In [68]:
coupon_item = coupon_item.merge(item_data, how='left', on='item_id')
coupon_item.head()

,coupon_id,item_id,brand,brand_type,category
0,105,37,56,0,Grocery
1,107,75,56,0,Grocery
2,494,76,209,1,Grocery
3,522,77,278,1,Grocery
4,518,77,278,1,Grocery


In [69]:
coupon_item_index = coupon_item.set_index('coupon_id')
coupon_item_index.info()

<class 'pandas.core.frame.DataFrame'>
Index: 92663 entries, 105 to 31
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   item_id     92663 non-null  int64 
 1   brand       92663 non-null  int64 
 2   brand_type  92663 non-null  int64 
 3   category    92663 non-null  object
dtypes: int64(3), object(1)
memory usage: 3.5+ MB


no null values  in "coupon_item_index" Dataset

### Train data

In [70]:
train_data = pd.read_csv(r'C:\Users\sahil\OneDrive\Pictures\Documents\OneDrive\Desktop\Project\Data\train1.csv')
train_data.head()

,id,campaign_id,coupon_id,customer_id,redemption_status
0,1,13,27,1053,0
1,2,13,116,48,0
2,6,9,635,205,0
3,7,13,644,1050,0
4,9,8,1017,1489,0


In [71]:
train_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 78369 entries, 0 to 78368
Data columns (total 5 columns):
 #   Column             Non-Null Count  Dtype
---  ------             --------------  -----
 0   id                 78369 non-null  int64
 1   campaign_id        78369 non-null  int64
 2   coupon_id          78369 non-null  int64
 3   customer_id        78369 non-null  int64
 4   redemption_status  78369 non-null  int64
dtypes: int64(5)
memory usage: 3.0 MB


no null values in "train_data" Dataset

### Test data

In [72]:
test_data = pd.read_csv(r'C:\Users\sahil\OneDrive\Pictures\Documents\OneDrive\Desktop\Project\Data\test1.csv')
test_data.head()

,id,campaign_id,coupon_id,customer_id
0,3,22,869,967
1,4,20,389,1566
2,5,22,981,510
3,8,25,1069,361
4,10,17,498,811


In [73]:
test_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50226 entries, 0 to 50225
Data columns (total 4 columns):
 #   Column       Non-Null Count  Dtype
---  ------       --------------  -----
 0   id           50226 non-null  int64
 1   campaign_id  50226 non-null  int64
 2   coupon_id    50226 non-null  int64
 3   customer_id  50226 non-null  int64
dtypes: int64(4)
memory usage: 1.5 MB


no null values in "test_data " Dataset


# Data Preprocessing

#### Combine train and test data

In [74]:
columns = train_data.columns[train_data.columns != 'redemption_status']
total_data = pd.concat([train_data[columns], test_data], sort=True)
total_data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 128595 entries, 0 to 50225
Data columns (total 4 columns):
 #   Column       Non-Null Count   Dtype
---  ------       --------------   -----
 0   campaign_id  128595 non-null  int64
 1   coupon_id    128595 non-null  int64
 2   customer_id  128595 non-null  int64
 3   id           128595 non-null  int64
dtypes: int64(4)
memory usage: 4.9 MB


### Data Transform

In [75]:
total ={}
for col in item_data.columns:
    total[col] = item_data[col].nunique()
print(total)

{'item_id': 74066, 'brand': 5528, 'brand_type': 2, 'category': 19}


Transforming Coupon-Item mapping into coupon specific variables

In [76]:
coupon_data = coupon_item_index.groupby('coupon_id').agg({
    'item_id': ['nunique'],
    'brand': ['nunique', most_frequent, least_frequent, most_frequent_count, least_frequent_count],
    'brand_type': ['nunique', most_frequent, least_frequent, most_frequent_count, least_frequent_count],
    'category': ['nunique', most_frequent, least_frequent, most_frequent_count, least_frequent_count]
})
coupon_data.columns = ['c_unique_items', 'c_unique_brand', 'c_freq_brand', 'c_rare_brand', 
                       'c_items_freq_brand', 'c_items_rare_brand', 'c_unique_brandt', 'c_freq_brandt',
                       'c_rare_brandt', 'c_items_freq_brandt', 'c_items_rare_brandt', 
                       'c_unique_category', 'c_freq_category', 'c_rare_category', 'c_items_freq_category', 
                       'c_items_rare_category']
coupon_data['c_coverage_item'] = coupon_data['c_unique_items'] / total.get('item_id')
coupon_data['c_coverage_brand'] = coupon_data['c_unique_brand'] / total.get('brand')
coupon_data['c_coverage_brandt'] = coupon_data['c_unique_brandt'] / total.get('brand_type')
coupon_data['c_coverage_category'] = coupon_data['c_unique_category'] / total.get('category')
coupon_data.head()

,c_unique_items,c_unique_brand,c_freq_brand,c_rare_brand,c_items_freq_brand,c_items_rare_brand,c_unique_brandt,c_freq_brandt,c_rare_brandt,c_items_freq_brandt,c_items_rare_brandt,c_unique_category,c_freq_category,c_rare_category,c_items_freq_category,c_items_rare_category,c_coverage_item,c_coverage_brand,c_coverage_brandt,c_coverage_category
coupon_id,,,,,,,,,,,,,,,,,,,,
1,39,3,1475,4700,25,5,1,1,1,39,39,2,Natural Products,Grocery,30,9,0.000527,0.000543,0.5,0.105263
2,2,1,2084,2084,2,2,1,1,1,2,2,1,Grocery,Grocery,2,2,0.000027,0.000181,0.5,0.052632
3,17,2,278,1558,12,5,1,1,1,17,17,1,Grocery,Grocery,17,17,0.000230,0.000362,0.5,0.052632
4,24,1,544,544,24,24,1,1,1,24,24,1,Grocery,Grocery,24,24,0.000324,0.000181,0.5,0.052632
5,7,1,5357,5357,7,7,1,1,1,7,7,1,Pharmaceutical,Pharmaceutical,7,7,0.000095,0.000181,0.5,0.052632


Transforming Customer Transactions into Customer Buying habbit

In [77]:
transaction_data['total_discount'] = transaction_data['coupon_discount'] + transaction_data['other_discount']
transaction_data['buying_price'] = transaction_data['selling_price'] + transaction_data['other_discount']
transaction_data['selling_price_pq'] = transaction_data['selling_price'] / transaction_data['quantity']
transaction_data['other_discount_pq'] = transaction_data['other_discount'] / transaction_data['quantity']
transaction_data['coupon_discount_pq'] = transaction_data['coupon_discount'] / transaction_data['quantity']
transaction_data['total_discount_pq'] = transaction_data['coupon_discount_pq'] + transaction_data['other_discount_pq']
transaction_data['buying_price_pq'] = transaction_data['selling_price_pq'] + transaction_data['other_discount_pq']
transaction_data['date'] = pd.to_datetime(transaction_data['date'])
transaction_data = transaction_data.merge(item_data, on='item_id', how='left')
transaction_data = transaction_data.set_index(['customer_id','date']).sort_index()
transaction_data.head()

item_id  quantity  selling_price  other_discount  \
customer_id date                                                           
1           2012-02-21     4953         1         142.12            0.00   
            2012-02-21     5626         1         106.50            0.00   
            2012-02-21     7808         1          99.38          -42.74   
            2012-02-21     8107         1         256.11          -46.31   
            2012-02-21     8307         1          89.05          -17.45   

                        coupon_discount  total_discount  buying_price  \
customer_id date                                                        
1           2012-02-21              0.0            0.00        142.12   
            2012-02-21              0.0            0.00        106.50   
            2012-02-21              0.0          -42.74         56.64   
            2012-02-21              0.0          -46.31        209.80   
            2012-02-21              0.0          -17.45         71.60   

                        selling_price_pq  other_discount_pq  \
customer_id date                                              
1           2012-02-21            142.12               0.00   
            2012-02-21            106.50               0.00   
            2012-02-21             99.38             -42.74   
            2012-02-21            256.11             -46.31   
            2012-02-21             89.05             -17.45   

                        coupon_discount_pq  total_discount_pq  \
customer_id date                                                
1           2012-02-21                 0.0               0.00   
            2012-02-21                 0.0               0.00   
            2012-02-21                 0.0             -42.74   
            2012-02-21                 0.0             -46.31   
            2012-02-21                 0.0             -17.45   

                        buying_price_pq  brand  brand_type category  
customer_id date                                                     
1           2012-02-21           142.12   1061           1  Grocery  
            2012-02-21           106.50    278           1  Grocery  
            2012-02-21            56.64     56           0  Grocery  
            2012-02-21           209.80    914           1  Grocery  
            2012-02-21            71.60    141           1  Grocery

In [78]:
transaction_data.info()

<class 'pandas.core.frame.DataFrame'>
MultiIndex: 1324566 entries, (1, Timestamp('2012-02-21 00:00:00')) to (1582, Timestamp('2013-06-25 00:00:00'))
Data columns (total 15 columns):
 #   Column              Non-Null Count    Dtype  
---  ------              --------------    -----  
 0   item_id             1324566 non-null  int64  
 1   quantity            1324566 non-null  int64  
 2   selling_price       1324566 non-null  float64
 3   other_discount      1324566 non-null  float64
 4   coupon_discount     1324566 non-null  float64
 5   total_discount      1324566 non-null  float64
 6   buying_price        1324566 non-null  float64
 7   selling_price_pq    1324566 non-null  float64
 8   other_discount_pq   1324566 non-null  float64
 9   coupon_discount_pq  1324566 non-null  float64
 10  total_discount_pq   1324566 non-null  float64
 11  buying_price_pq     1324566 non-null  float64
 12  brand               1324566 non-null  int64  
 13  brand_type          1324566 non-null  int64  
 1

In [79]:
transaction_data.describe()

,item_id,quantity,selling_price,other_discount,coupon_discount,total_discount,buying_price,selling_price_pq,other_discount_pq,coupon_discount_pq,total_discount_pq,buying_price_pq,brand,brand_type
count,1.324566e+06,1.324566e+06,1.324566e+06,1.324566e+06,1.324566e+06,1.324566e+06,1.324566e+06,1.324566e+06,1.324566e+06,1.324566e+06,1.324566e+06,1.324566e+06,1.324566e+06,1.324566e+06
mean,2.951903e+04,1.306633e+02,1.146036e+02,-1.776871e+01,-5.948983e-01,-1.836360e+01,9.683493e+01,8.770231e+01,-1.355429e+01,-4.759791e-01,-1.403027e+01,7.414802e+01,8.840647e+02,7.308024e-01
std,1.790806e+04,1.311545e+03,1.529053e+02,3.788867e+01,7.069367e+00,3.929699e+01,1.481945e+02,9.287411e+01,2.575621e+01,5.405928e+00,2.655212e+01,9.111477e+01,1.125452e+03,4.435430e-01
min,1.000000e+00,1.000000e+00,3.600000e-01,-3.120310e+03,-1.992230e+03,-3.258160e+03,-1.424440e+03,5.911488e-02,-2.671500e+03,-1.068240e+03,-2.671500e+03,-1.424440e+03,1.000000e+00,0.000000e+00
25%,1.468400e+04,1.000000e+00,4.916000e+01,-2.315000e+01,0.000000e+00,-2.458000e+01,3.526000e+01,3.562000e+01,-1.781000e+01,0.000000e+00,-1.781000e+01,2.743000e+01,5.600000e+01,0.000000e+00
50%,2.659700e+04,1.000000e+00,7.801000e+01,-1.780000e+00,0.000000e+00,-3.210000e+00,6.447000e+01,7.088000e+01,-3.560459e-03,0.000000e+00,-7.125000e-01,5.308000e+01,5.330000e+02,1.000000e+00
75%,4.240575e+04,1.000000e+00,1.243100e+02,0.000000e+00,0.000000e+00,0.000000e+00,1.068600e+02,1.065000e+02,0.000000e+00,0.000000e+00,0.000000e+00,9.582000e+01,1.124000e+03,1.000000e+00
max,7.406600e+04,8.963800e+04,1.780964e+04,0.000000e+00,0.000000e+00,0.000000e+00,1.602864e+04,1.780964e+04,0.000000e+00,0.000000e+00,0.000000e+00,1.602864e+04,5.528000e+03,1.000000e+00


In [80]:
customer_history = transaction_data.groupby('customer_id').agg({
    'item_id': ['nunique', 'count'],
    'quantity': 'sum',
    'selling_price': 'mean',
    'buying_price': 'mean',
    'other_discount': 'mean',
    'coupon_discount': 'mean',
    'total_discount': 'mean',
    'selling_price_pq': 'mean',
    'buying_price_pq': 'mean',
    'other_discount_pq': 'mean',
    'coupon_discount_pq': 'mean',
    'total_discount_pq': 'mean',
    'brand': ['nunique', most_frequent, least_frequent, most_frequent_count, least_frequent_count],
    'brand_type': ['nunique', most_frequent, least_frequent, most_frequent_count, least_frequent_count],
    'category': ['nunique', most_frequent, least_frequent, most_frequent_count, least_frequent_count]
})
customer_history.columns = ['overall_unique_items', 'overall_items', 'overall_quantity', 'overall_sprice', 'overall_bprice', 'overall_odiscount', 'overall_cdiscount', 'overall_tdiscount', 'overall_sprice_pq', 'overall_bprice_pq', 'overall_odiscount_pq', 'overall_cdiscount_pq', 'overall_tdiscount_pq', 'overall_unique_brand', 'overall_freq_brand', 'overall_rare_brand', 'overall_items_freq_brand', 'overall_items_rare_brand', 'overall_unique_brandt', 'overall_freq_brandt', 'overall_rare_brandt', 'overall_items_freq_brandt', 'overall_items_rare_brandt', 'overall_unique_category', 'overall_freq_category', 'overall_rare_category', 'overall_items_freq_category', 'overall_items_rare_category']
customer_history['overall_coverage_item'] = customer_history['overall_unique_items'] / total.get("item_id")
customer_history['overall_coverage_brand'] = customer_history['overall_unique_brand'] / total.get("brand")
customer_history['overall_coverage_brandt'] = customer_history['overall_unique_brandt'] / total.get("brand_type")
customer_history['overall_coverage_category'] = customer_history['overall_unique_category'] / total.get("category")
customer_history['overall_podiscount'] = customer_history['overall_odiscount'] / customer_history['overall_bprice']
customer_history['overall_pcdiscount'] = customer_history['overall_cdiscount'] / customer_history['overall_bprice']
customer_history['overall_ptdiscount'] = customer_history['overall_tdiscount'] / customer_history['overall_bprice']
customer_history['overall_podiscount_pq'] = customer_history['overall_odiscount_pq'] / customer_history['overall_bprice_pq']
customer_history['overall_pcdiscount_pq'] = customer_history['overall_cdiscount_pq'] / customer_history['overall_bprice_pq']
customer_history['overall_ptdiscount_pq'] = customer_history['overall_tdiscount_pq'] / customer_history['overall_bprice_pq']
customer_history = customer_history.reset_index()
customer_history.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1582 entries, 0 to 1581
Data columns (total 39 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   customer_id                  1582 non-null   int64  
 1   overall_unique_items         1582 non-null   int64  
 2   overall_items                1582 non-null   int64  
 3   overall_quantity             1582 non-null   int64  
 4   overall_sprice               1582 non-null   float64
 5   overall_bprice               1582 non-null   float64
 6   overall_odiscount            1582 non-null   float64
 7   overall_cdiscount            1582 non-null   float64
 8   overall_tdiscount            1582 non-null   float64
 9   overall_sprice_pq            1582 non-null   float64
 10  overall_bprice_pq            1582 non-null   float64
 11  overall_odiscount_pq         1582 non-null   float64
 12  overall_cdiscount_pq         1582 non-null   float64
 13  overall_tdiscount_

In [81]:
customer_history.head(10)

,customer_id,overall_unique_items,overall_items,overall_quantity,overall_sprice,overall_bprice,overall_odiscount,overall_cdiscount,overall_tdiscount,overall_sprice_pq,...,overall_coverage_item,overall_coverage_brand,overall_coverage_brandt,overall_coverage_category,overall_podiscount,overall_pcdiscount,overall_ptdiscount,overall_podiscount_pq,overall_pcdiscount_pq,overall_ptdiscount_pq
0,1,463,1048,1227,94.001842,77.751460,-16.250382,-2.019876,-18.270258,84.632943,...,0.006251,0.029486,1.0,0.473684,-0.209004,-0.025979,-0.234983,-0.178807,-0.024361,-0.203168
1,2,352,419,474,102.864033,86.033604,-16.830430,-0.595084,-17.425513,94.373588,...,0.004753,0.027677,1.0,0.473684,-0.195626,-0.006917,-0.202543,-0.165950,-0.005602,-0.171551
2,3,406,705,8163,103.617404,80.903177,-22.714227,-3.091546,-25.805773,71.007598,...,0.005482,0.020622,1.0,0.421053,-0.280758,-0.038213,-0.318971,-0.247219,-0.033123,-0.280342
3,4,125,220,280,154.423727,141.118318,-13.305409,-0.404773,-13.710182,129.373114,...,0.001688,0.013025,1.0,0.421053,-0.094285,-0.002868,-0.097154,-0.073730,-0.000672,-0.074402
4,5,490,792,93353,130.827146,117.169230,-13.657917,-0.114684,-13.772601,104.222145,...,0.006616,0.030391,1.0,0.736842,-0.116566,-0.000979,-0.117545,-0.121133,-0.001234,-0.122367
5,6,429,583,707,102.072419,90.070480,-12.001938,-0.702607,-12.704545,91.470758,...,0.005792,0.025687,1.0,0.473684,-0.133251,-0.007801,-0.141051,-0.127232,-0.007812,-0.135043
6,7,780,1053,57763,102.067455,84.263656,-17.803799,-0.745897,-18.549696,86.887374,...,0.010531,0.039436,1.0,0.684211,-0.211287,-0.008852,-0.220139,-0.196325,-0.008989,-0.205314
7,8,719,1318,1334454,227.224697,203.695751,-23.528945,-5.011601,-28.540546,107.760461,...,0.009708,0.035094,1.0,0.631579,-0.115510,-0.024603,-0.140114,-0.158790,-0.039974,-0.198763
8,9,405,557,117820,102.829210,81.727325,-21.101885,-0.808312,-21.910197,73.541581,...,0.005468,0.021165,1.0,0.578947,-0.258199,-0.009890,-0.268089,-0.292885,-0.013592,-0.306477
9,10,268,491,81165,103.788798,92.309246,-11.479552,0.000000,-11.479552,84.688004,...,0.003618,0.020080,1.0,0.526316,-0.124360,0.000000,-0.124360,-0.132779,0.000000,-0.132779


### Merge Dataset

In [82]:
total_data = total_data.merge(campaign_data, on='campaign_id', how='left')
total_data = total_data.merge(customer_data, on='customer_id', how='left')
total_data = total_data.merge(coupon_data, on='coupon_id', how='left')
total_data = total_data.merge(customer_history, on='customer_id', how='left')
total_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 128595 entries, 0 to 128594
Data columns (total 72 columns):
 #   Column                       Non-Null Count   Dtype         
---  ------                       --------------   -----         
 0   campaign_id                  128595 non-null  int64         
 1   coupon_id                    128595 non-null  int64         
 2   customer_id                  128595 non-null  int64         
 3   id                           128595 non-null  int64         
 4   campaign_type                128595 non-null  int64         
 5   start_date                   128595 non-null  datetime64[ns]
 6   end_date                     128595 non-null  datetime64[ns]
 7   duration                     128595 non-null  int64         
 8   age_range                    74600 non-null   object        
 9   marital_status               74600 non-null   float64       
 10  rented                       74600 non-null   float64       
 11  family_size               

In [83]:
range_start_delta = pd.Timedelta('223 days')
range_end_delta = pd.Timedelta('110 days')

def extract_range_features(row):
    start_time = pd.Timestamp(campaign_data_index.loc[row.campaign_id].start_date)
    range_start, range_end = start_time - range_start_delta, start_time - range_end_delta
    items = pd.Series(coupon_item_index.loc[row.coupon_id].item_id)

    overall_history = transaction_data.loc[row.customer_id]
    history = overall_history[range_start:range_end]
    overall_coupon_history = overall_history[overall_history.item_id.isin(items)]
    coupon_history = history[history.item_id.isin(items)]

    for df_name, df in [('overall_coupon_', overall_coupon_history), ('range_', history), ('range_coupon_', coupon_history)]:
        row[df_name+'unique_items'] = df['item_id'].nunique()
        row[df_name+'items'] = df['item_id'].count()
        row[df_name+'quantity'] = df['quantity'].sum()
        
        for new_name, old_name in [('sprice', 'selling_price'), ('bprice', 'buying_price'), ('odiscount', 'other_discount'), ('cdiscount', 'coupon_discount'), ('tdiscount', 'total_discount'), ('sprice_pq', 'selling_price_pq'), ('bprice_pq', 'buying_price_pq'), ('odiscount_pq', 'other_discount_pq'), ('cdiscount_pq', 'coupon_discount_pq'), ('tdiscount_pq', 'total_discount_pq')]:
            row[df_name+''+new_name] = df[old_name].mean()
        
        for name in ['odiscount', 'cdiscount', 'tdiscount']:
            bprice = row[df_name+'bprice']
            row[df_name+'p'+name] = row[df_name+''+name] / bprice if bprice > 0 else np.NaN
            
        for name in ['odiscount_pq', 'cdiscount_pq', 'tdiscount_pq']:
            bprice_pq = row[df_name+'bprice_pq']
            row[df_name+'p'+name] = row[df_name+''+name] / bprice_pq if bprice_pq > 0 else np.NaN
        
        for new_name, old_name in [('brand', 'brand'), ('brandt', 'brand_type'), ('category','category')]:
            unique_count = df[old_name].nunique()
            row[df_name+'unique_'+new_name] = unique_count
            
            if unique_count == 0:
                row[df_name+'items_freq_'+new_name] = 0
                row[df_name+'items_rare_'+new_name] = 0
                row[df_name+'freq_'+new_name] = np.NaN
                row[df_name+'rare_'+new_name] = np.NaN
            else:
                freq = most_frequent(df[old_name])
                row[df_name+'freq_'+new_name] = freq
                row[df_name+'items_freq_'+new_name] = df[df[old_name] == freq]['item_id'].nunique()
                row[df_name+'items_p_freq_'+new_name] = row[df_name+'items_freq_'+new_name] / df['item_id'].nunique()

                rare = least_frequent(df[old_name])
                row[df_name+'rare_'+new_name] = rare
                row[df_name+'items_rare_'+new_name] = df[df[old_name] == rare]['item_id'].nunique()
                row[df_name+'items_p_rare_'+new_name] = row[df_name+'items_rare_'+new_name] / df['item_id'].nunique()
            
        row[df_name+'coverage_item'] = row[df_name+'unique_items'] / total_items
        row[df_name+'coverage_brand'] = row[df_name+'unique_brand'] / total_brands
        row[df_name+'coverage_brandt'] = row[df_name+'unique_brandt'] / total_brand_types
        row[df_name+'coverage_category'] = row[df_name+'unique_category'] / total_categories
    
    if coupon_history.shape[0] > 0:
        row['redemption_ratio'] = coupon_history[coupon_history['coupon_discount'] < 0].shape[0] / coupon_history.shape[0]
    else:
        row['redemption_ratio'] = np.NaN
    
    if history.shape[0] > 0:
        row['overall_redemption_ratio'] = history[history['coupon_discount'] < 0].shape[0] / history.shape[0]
    else:
        row['overall_redemption_ratio'] = np.NaN
        
    return row

def apply_range_features(df):
    return df.apply(extract_range_features, axis=1)

In [84]:
#total_data = parallelize_dataframe(total_data, apply_range_features, n_cores=4)
#total_data = apply_range_features(total_data.head(5))
total_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 128595 entries, 0 to 128594
Data columns (total 72 columns):
 #   Column                       Non-Null Count   Dtype         
---  ------                       --------------   -----         
 0   campaign_id                  128595 non-null  int64         
 1   coupon_id                    128595 non-null  int64         
 2   customer_id                  128595 non-null  int64         
 3   id                           128595 non-null  int64         
 4   campaign_type                128595 non-null  int64         
 5   start_date                   128595 non-null  datetime64[ns]
 6   end_date                     128595 non-null  datetime64[ns]
 7   duration                     128595 non-null  int64         
 8   age_range                    74600 non-null   object        
 9   marital_status               74600 non-null   float64       
 10  rented                       74600 non-null   float64       
 11  family_size               

#### Derive new variables

In [85]:
# Deriving more variables from existing
#for column in ['unique_items', 'items', 'quantity', 'sprice', 'bprice', 'odiscount', 'cdiscount', 'tdiscount', 'podiscount', 'pcdiscount', 'ptdiscount', 'sprice_pq', 'bprice_pq', 'odiscount_pq', 'cdiscount_pq', 'tdiscount_pq', 'podiscount_pq', 'pcdiscount_pq', 'ptdiscount_pq', 'unique_brand', 'unique_brandt', 'unique_category', 'coverage_brand', 'coverage_category']:
    #total_data['diff_overall_'+column] = total_data['overall_'+column] - total_data['overall_coupon_'+column]
    #total_data['diff_range_'+column] = total_data['range_'+column] - total_data['range_coupon_'+column]
    #total_data['diff_coupon_'+column] = total_data['overall_coupon_'+column] - total_data['range_coupon_'+column]
    
    #total_data['diff_p_overall_'+column] = total_data['diff_overall_'+column] / total_data['overall_'+column]
    #total_data['diff_p_range_'+column] = total_data['diff_range_'+column] / total_data['range_'+column]
    #total_data['diff_p_coupon_'+column] = total_data['diff_coupon_'+column] / total_data['overall_coupon_'+column]

#for column in ['unique_items', 'unique_brand', 'unique_brandt', 'unique_category', 'coverage_item', 'coverage_brand', 'coverage_brandt', 'coverage_category']:
   # total_data['c_diff_'+column] = total_data['c_'+column] - total_data['range_coupon_'+column]
    
#for column in ['freq_brand', 'rare_brand', 'freq_brandt', 'rare_brandt', 'freq_category', 'rare_category']:
   # total_data['match_'+column] = (total_data['c_'+column] == total_data['range_coupon_'+column]).astype('int')
    
#total_data.info()

 #### Splitting total data into train and test

In [86]:
test_data = test_data[['id']].merge(total_data, on='id', how='left')
test_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50226 entries, 0 to 50225
Data columns (total 72 columns):
 #   Column                       Non-Null Count  Dtype         
---  ------                       --------------  -----         
 0   id                           50226 non-null  int64         
 1   campaign_id                  50226 non-null  int64         
 2   coupon_id                    50226 non-null  int64         
 3   customer_id                  50226 non-null  int64         
 4   campaign_type                50226 non-null  int64         
 5   start_date                   50226 non-null  datetime64[ns]
 6   end_date                     50226 non-null  datetime64[ns]
 7   duration                     50226 non-null  int64         
 8   age_range                    30939 non-null  object        
 9   marital_status               30939 non-null  float64       
 10  rented                       30939 non-null  float64       
 11  family_size                  30939 non-nu

In [90]:
test_data.to_csv(r'C:\Users\sahil\OneDrive\Pictures\Documents\OneDrive\Desktop\Project\Data\test.csv', index=False)

In [91]:
train_data = train_data[['id','redemption_status']].merge(total_data, on='id', how='left')
train_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 78369 entries, 0 to 78368
Data columns (total 73 columns):
 #   Column                       Non-Null Count  Dtype         
---  ------                       --------------  -----         
 0   id                           78369 non-null  int64         
 1   redemption_status            78369 non-null  int64         
 2   campaign_id                  78369 non-null  int64         
 3   coupon_id                    78369 non-null  int64         
 4   customer_id                  78369 non-null  int64         
 5   campaign_type                78369 non-null  int64         
 6   start_date                   78369 non-null  datetime64[ns]
 7   end_date                     78369 non-null  datetime64[ns]
 8   duration                     78369 non-null  int64         
 9   age_range                    43661 non-null  object        
 10  marital_status               43661 non-null  float64       
 11  rented                       43661 non-nu

In [92]:
train_data.to_csv(r'C:\Users\sahil\OneDrive\Pictures\Documents\OneDrive\Desktop\Project\Data\train1.csv', index=False)
train_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 78369 entries, 0 to 78368
Data columns (total 73 columns):
 #   Column                       Non-Null Count  Dtype         
---  ------                       --------------  -----         
 0   id                           78369 non-null  int64         
 1   redemption_status            78369 non-null  int64         
 2   campaign_id                  78369 non-null  int64         
 3   coupon_id                    78369 non-null  int64         
 4   customer_id                  78369 non-null  int64         
 5   campaign_type                78369 non-null  int64         
 6   start_date                   78369 non-null  datetime64[ns]
 7   end_date                     78369 non-null  datetime64[ns]
 8   duration                     78369 non-null  int64         
 9   age_range                    43661 non-null  object        
 10  marital_status               43661 non-null  float64       
 11  rented                       43661 non-nu

# Summary

1. Customer Information<br>
    a. There were many missing values in "no_of_children" and "marital_status"<br>
    b. Assuming missing no_of_children as zero childrens<br>
    c. Marital Status was calculated using family_size and no_of_children<br>
2. Train and Test data contains many customer whose information is not available
3. Derived new variables from coupons and customer transactions
4. Merged all other data with train and test dataset
5. Derived new variables related to change is customer behaviors and matching with coupon data
6. There are lots of Outliers present in the dataset, model must learn that in order to predict the redemption